In [ ]:
# Import the libraries

import argparse
import os.path as ops
import time
import utils
import sys
import cv2
import glog as log
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import math
import copy
from control import matlab
from config import global_config
from lanenet_model import lanenet
from lanenet_model import lanenet_postprocess
import time
import cvxopt
cvxopt.solvers.options['show_progress'] = False
import termios
import keyboard
import serial
import struct
import numpy as np
from xbee import XBee
import pyzed.sl as sl

CFG = global_config.cfg

mid_line_window = np.zeros((720, 2))

In [ ]:
def init_args():
    
    """
    :return:
    """
    
    parser = argparse.ArgumentParser()
    parser.add_argument('--image_path', type=str, help='The image path or the src image save dir')
    parser.add_argument('--weights_path', type=str, help='The model weights path')

    return parser.parse_args()


In [ ]:
def args_str2bool(arg_value):

    
    """
    :param arg_value:
    :return:
    """
    
    if arg_value.lower() in ('yes', 'true', 't', 'y', '1'):
        return True

    elif arg_value.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Unsupported value encountered.')
        

In [ ]:
def minmax_scale(input_arr):
    
    """
    :param input_arr:
    :return:
    """
    
    min_val = np.min(input_arr)
    max_val = np.max(input_arr)

    output_arr = (input_arr - min_val) * 255.0 / (max_val - min_val)

    return output_arr


In [ ]:

class system():

"""
A system class has been created which represents a discrete time system.
It consists of the following parameters.
Params:
state space parameters of the system A, B, C, D
The states of the system x
The control input of the system u
returns:
"""
    
    def __init__(self, A, B, C, D = None, init_states = None):
        
        self.A = A
        self.B = B
        self.C = C
        
        if D is not None:
            self.D = D
        
        self.x = np.zeros(self.A.shape[0])
        if init_states is not None:
            self.x = np.copy(init_states)
   
        self.his_x = [init_states]

        
        
    def update_state(self, u, dt = 0.01):
        self.temp_x = self.x.reshape(-1,1)
        self.temp_u = u.reshape(-1,1)
    
        # Calculating the coefficients for Range Kutta Method
        f1 = dt*(self.A.dot(self.temp_x)+self.B.dot(self.temp_u))
        f2 = dt*(self.A.dot(self.temp_x+f1/2)+ self.B.dot(self.temp_u))
        f3 = dt*(self.A.dot(self.temp_x+f2/2)+ self.B.dot(self.temp_u))
        f4 = dt*(self.A.dot(self.temp_x+f3)+ self.B.dot(self.temp_u))
        self.x = self.x + (f1+2*f2+2*f3+f4)/6
        temp_state = np.array(self.x)
        self.his_x.append(temp_state)

In [ ]:

def mpc_calculation(A, B, Q, R, x0, horizon, u_len, u_max, x_max, x_des):
    
"""This function calculates the control input using the system parameters.
A sparse matrix is created to increase the speed of computation
params : 
A, B = system parameters
"""

    num_states = x0.size*horizon
    num_control = u_len*horizon
    num_vars = num_states+num_control
    
    # Contruction of required martrices
    Q1 = np.zeros([num_vars, num_vars])
    A1 = np.zeros([num_states, num_vars])
    b1 = np.zeros([num_states])
    b1[:x0.size] = -A.dot(x0).flatten()  
    
    q1 = np.zeros([num_vars])
    
    for i in range(horizon):
        Q1[x0.size*i:x0.size*(i+1), x0.size*i:x0.size*(i+1)] = Q
        Q1[num_states+u_len*i:num_states+u_len*(i+1),
           num_states+u_len*i:num_states+u_len*(i+1)] = R
        
        q1[x0.size*i:x0.size*(i+1)] = -x_des[:,i].dot(Q)
        A1[x0.size*i:x0.size*(i+1), num_states+u_len*i:num_states+u_len*(i+1)] = B
        if i>0:
            A1[x0.size*i:x0.size*(i+1), x0.size*(i-1):x0.size*(i+1)] = np.hstack((A, -np.eye(x0.size)))
        else:
            A1[x0.size*i:x0.size*(i+1), x0.size*(i):x0.size*(i+1)] = -np.eye(x0.size)
            
    G = np.zeros([2*num_control, num_vars])
    G[:, num_states:] = np.vstack((np.eye(num_control), -np.eye(num_control)))
    G = np.vstack((G, np.eye(num_states, num_vars)))
    G = np.vstack((G, -np.eye(num_states, num_vars)))
    h = np.vstack((np.tile(u_max, horizon), np.tile(u_max, horizon))).flatten()
    tt = np.vstack((np.tile(x_max, horizon), np.tile(x_max, horizon))).flatten()
    h = np.hstack((h.T, tt.T)).astype(float)
    Q = cvxopt.matrix(Q1)
    p = cvxopt.matrix(q1)
    G = cvxopt.matrix(G)
    h = cvxopt.matrix(h)
    A = cvxopt.matrix(A1)
    b = cvxopt.matrix(b1)
    sol = cvxopt.solvers.qp(Q, p, G, h, A, b)
    x = np.array(sol['x'])
    u = x[num_states:num_states+u_len]
    return u


In [ ]:
def set_transmission():
    port = '/dev/ttyUSB0'
    serial_port = serial.Serial(port,9600)
    serial_port.close()
    serial_port.open()
    # time.sleep(0.05)
    print("Transmission Set up")
    return serial_port

In [ ]:
def transmission_function(u, serial_port):
    serial_port.write(struct.pack('>B',3))
    serial_port.write(struct.pack('>B',u))
    print("transmitted {0}" .format(u))

In [ ]:
def stop_car(serial_port):
    serial_port.write(struct.pack('>B',1))
    serial_port.write(struct.pack('>B',128+30))

In [ ]:
camera_settings = sl.CAMERA_SETTINGS.CAMERA_SETTINGS_BRIGHTNESS
str_camera_settings = "BRIGHTNESS"
step_camera_settings = 1


# Lane detection
# Some global variables
polyfit_left=None
polyfit_right=None

past_good_left_lines = []
past_good_right_lines = []

running_mean_difference_between_lines = 0

def get_line_predictions(non_zeros_x, non_zeros_y, left_coordinates, right_coordinates, num_rows):
    """
        Given ncoordinates of non-zeros pixels and coordinates of non-zeros pixels within the sliding windows,
        this function generates a prediction for the lane line.
    """
    left_x = non_zeros_x[left_coordinates]
    left_y = non_zeros_y[left_coordinates]

    # If no pixels were found return None
    if(left_y.size == 0 or left_x.size == 0):
        return None, None

    # Fit the polynomial
    polyfit_left = np.polyfit(left_y, left_x, 2)

    right_x = non_zeros_x[right_coordinates]
    right_y = non_zeros_y[right_coordinates]

    # If no pixels were found return None
    if(right_y.size == 0 or right_x.size == 0):
        return None, None

    # Fit the polynomial
#     print(right_x.shape,right_y.shape)
    polyfit_right = np.polyfit(right_y, right_x, 2)

    # If no pixels were found return None
    y_points = np.linspace(0, num_rows-1, num_rows)

    # Generate the lane lines from the polynomial fit
    left_x_predictions = polyfit_left[0]*y_points**2 + polyfit_left[1]*y_points + polyfit_left[2]
    right_x_predictions = polyfit_right[0]*y_points**2 + polyfit_right[1]*y_points + polyfit_right[2]

    return left_x_predictions, right_x_predictions


In [ ]:
def brute_search(warped):
    """
        This function searches for lane lines from scratch.
        Thresholding & performing a sliding window search.
    """
    non_zeros = warped.nonzero()
    non_zeros_y = non_zeros[0]
    non_zeros_x = non_zeros[1]

    num_rows = warped.shape[0]

    histogram = np.sum(warped[warped.shape[0]//2:,:], axis=0)

    half_width = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:half_width])
    rightx_base = np.argmax(histogram[half_width:]) + half_width

    num_windows = 10
    window_height = np.int(num_rows/num_windows)
    window_half_width = 50

    min_pixels = 100

    left_coordinates = []
    right_coordinates = []

    for window in range(num_windows):
        y_max = num_rows - window*window_height
        y_min = num_rows - (window+1)* window_height

        left_x_min = leftx_base - window_half_width
        left_x_max = leftx_base + window_half_width

        good_left_window_coordinates = ((non_zeros_x >= left_x_min) & (non_zeros_x <= left_x_max) & (non_zeros_y >= y_min) & (non_zeros_y <= y_max)).nonzero()[0]
        left_coordinates.append(good_left_window_coordinates)

        if len(good_left_window_coordinates) > min_pixels:
            leftx_base = np.int(np.mean(non_zeros_x[good_left_window_coordinates]))

        right_x_min = rightx_base - window_half_width
        right_x_max = rightx_base + window_half_width

        good_right_window_coordinates = ((non_zeros_x >= right_x_min) & (non_zeros_x <= right_x_max) & (non_zeros_y >= y_min) & (non_zeros_y <= y_max)).nonzero()[0]
        right_coordinates.append(good_right_window_coordinates)

        if len(good_right_window_coordinates) > min_pixels:
            rightx_base = np.int(np.mean(non_zeros_x[good_right_window_coordinates]))

    left_coordinates = np.concatenate(left_coordinates)
    right_coordinates = np.concatenate(right_coordinates)

    left_x_predictions, right_x_predictions = get_line_predictions(non_zeros_x, non_zeros_y, left_coordinates, right_coordinates, num_rows)
    return left_x_predictions, right_x_predictions


In [1]:
def get_averaged_line(previous_lines, new_line):
    """
        This function computes an averaged lane line by averaging over previous good frames.
    """

    # Number of frames to average over
    num_frames = 12

    if new_line is None:
        # No line was detected

        if len(previous_lines) == 0:
            # If there are no previous lines, return None
            return previous_lines, None
        else:
            # Else return the last line
            return previous_lines, previous_lines[-1]
    else:
        if len(previous_lines) < num_frames:
            # we need at least num_frames frames to average over
            previous_lines.append(new_line)
            return previous_lines, new_line
        else:
            # average over the last num_frames frames
            previous_lines[0:num_frames-1] = previous_lines[1:]
            previous_lines[num_frames-1] = new_line
            new_line = np.zeros_like(new_line)
            for i in range(num_frames):
                new_line += previous_lines[i]
            new_line /= num_frames
            return previous_lines, new_line


In [2]:
def get_mean_distance_between_lines(left_line, right_line, running_average):
    """
        Returns running weighted average of simple difference between left and right lines
    """
    mean_distance = np.mean(right_line - left_line)
    if running_average == 0:
        running_average = mean_distance
    else:
        running_average = 0.9*running_average + 0.1*mean_distance
    return running_average


In [3]:
def measure_radius_of_curvature(x_values, num_rows):
    ym_per_pix = 4.5/720 # meters per pixel in y dimension
    xm_per_pix = 2.8/700 # meters per pixel in x dimension
    # If no pixels were found return None
    y_points = np.linspace(0, num_rows-1, num_rows)
    y_eval = np.max(y_points)

    # Fit new polynomials to x,y in world space
#     print((y_points*ym_per_pix).shape,(x_values*xm_per_pix).shape)
    fit_cr = np.polyfit(y_points*ym_per_pix, x_values*xm_per_pix, 2)
    curverad = ((1 + (2*fit_cr[0]*y_eval*ym_per_pix + fit_cr[1])**2)**1.5) / np.absolute(2*fit_cr[0])
    return curverad


In [4]:
def interpolate(a1, a2, poly_deg=2, n_points=100, plot=True):

    min_a1_x, max_a1_x = min(a1[:,0]), max(a1[:,0])
    new_a1_x = np.linspace(min_a1_x, max_a1_x, n_points)
    a1_coefs = np.polyfit(a1[:,0],a1[:,1], poly_deg)
    new_a1_y = np.polyval(a1_coefs, new_a1_x)

    min_a2_x, max_a2_x = min(a2[:,0]), max(a2[:,0])
    new_a2_x = np.linspace(min_a2_x, max_a2_x, n_points)
    a2_coefs = np.polyfit(a2[:,0],a2[:,1], poly_deg)
    new_a2_y = np.polyval(a2_coefs, new_a2_x)

    midx = [np.mean([new_a1_x[i], new_a2_x[i]]) for i in range(n_points)]
    midy = [np.mean([new_a1_y[i], new_a2_y[i]]) for i in range(n_points)]

    if plot:
        plt.plot(a1[:,0], a1[:,1],c='black')
        plt.plot(a2[:,0], a2[:,1],c='black')
        plt.plot(midx, midy, '--', c='black')
        plt.show()

    return np.array([[x, y] for x, y in zip(midx, midy)])

In [5]:
def pipeline_final(img, thresh_img, M, M_inv):
    # global variables to store the polynomial coefficients of the line detected in the last frame
    global polyfit_right
    global polyfit_left

    # global variables to store the line coordinates in previous n (=4) frames
    global past_good_right_lines
    global past_good_left_lines

    # global variable which contains running average of the mean difference between left and right lanes
    global running_mean_difference_between_lines

    img_shape = img.shape

    img_size = (img_shape[1], img_shape[0])

    # get thresholded image
    thresholded = thresh_img # get_thresholded_image(img)

     # perform a perspective transform
    warped = cv2.warpPerspective(thresholded, M, img_size , flags=cv2.INTER_LINEAR)

    out_img = np.dstack((warped, warped, warped))*255

    non_zeros = warped.nonzero()
    non_zeros_y = non_zeros[0]
    non_zeros_x = non_zeros[1]

    num_rows = warped.shape[0]
    y_points = np.linspace(0, num_rows-1, num_rows)

    # at the very first frame when polynomials are not fitted
    if (polyfit_left is None) or (polyfit_right is None):
        # If the polynomial coefficients of the previous frames are None then perform a brute force search
        brute = True
        left_x_predictions, right_x_predictions = brute_search(warped)
    else:
        # Else search in a margin of 100 pixels on each side of the pervious polynomial fit
        brute = False
        margin = 100
        left_x_predictions = polyfit_left[0]*non_zeros_y**2 + polyfit_left[1]*non_zeros_y + polyfit_left[2]
        left_coordinates = ((non_zeros_x >= left_x_predictions - margin) & (non_zeros_x <= left_x_predictions + margin)).nonzero()[0]

        right_x_predictions = polyfit_right[0]*non_zeros_y**2 + polyfit_right[1]*non_zeros_y + polyfit_right[2]
        right_coordinates = ((non_zeros_x >= right_x_predictions - margin) & (non_zeros_x <= right_x_predictions + margin)).nonzero()[0]

        left_x_predictions, right_x_predictions = get_line_predictions(non_zeros_x, non_zeros_y, left_coordinates, right_coordinates, num_rows)

    if (left_x_predictions is None or right_x_predictions is None):
        if not brute:
            left_x_predictions, right_x_predictions = brute_search(warped)

    bad_lines = False

    if (left_x_predictions is None or right_x_predictions is None):
        bad_lines = True
    else:
        mean_difference = np.mean(right_x_predictions - left_x_predictions)

        if running_mean_difference_between_lines == 0:
            running_mean_difference_between_lines = mean_difference

        if (mean_difference < 0.7*running_mean_difference_between_lines or mean_difference > 1.3*running_mean_difference_between_lines):
            bad_lines = True
            if not brute:
                left_x_predictions, right_x_predictions = brute_search(warped)
                if (left_x_predictions is None or right_x_predictions is None):
                    bad_lines = True
                else:
                    mean_difference = np.mean(right_x_predictions - left_x_predictions)
                    if (mean_difference < 0.7*running_mean_difference_between_lines or mean_difference > 1.3*running_mean_difference_between_lines):
                        bad_lines = True
                    else:
                        bad_lines = False
        else:
            bad_lines = False

    if bad_lines:
        polyfit_left = None
        polyfit_right = None
        if len(past_good_left_lines) == 0 and len(past_good_right_lines) == 0:
            return img
        else:
            left_x_predictions = past_good_left_lines[-1]
            right_x_predictions = past_good_right_lines[-1]
    else:
        past_good_left_lines, left_x_predictions = get_averaged_line(past_good_left_lines, left_x_predictions)
        past_good_right_lines, right_x_predictions = get_averaged_line(past_good_right_lines, right_x_predictions)
        mean_difference = np.mean(right_x_predictions - left_x_predictions)
        running_mean_difference_between_lines = 0.9*running_mean_difference_between_lines + 0.1*mean_difference

    left_line_window = np.array(np.transpose(np.vstack([left_x_predictions, y_points])))
    right_line_window = np.array(np.flipud(np.transpose(np.vstack([right_x_predictions, y_points]))))

    # compute the radius of curvature
    left_curve_rad = measure_radius_of_curvature(left_x_predictions, num_rows)
    right_curve_rad = measure_radius_of_curvature(right_x_predictions, num_rows)
    average_curve_rad = (left_curve_rad + right_curve_rad)/2
    curvature_string = "Radius of curvature: %.2f m" % average_curve_rad

    # compute the offset from the center
    lane_center = (right_x_predictions[num_rows-1] + left_x_predictions[num_rows-1])/2
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    center_offset_pixels = abs(img_size[0]/2 - lane_center)
    center_offset_mtrs = xm_per_pix*center_offset_pixels
    offset_string = "Center offset: %.2f m" % center_offset_mtrs

    poly_points = np.vstack([left_line_window, right_line_window])


    cv2.fillPoly(out_img, np.int_([poly_points]), [0,255, 0])

    cv2.polylines(out_img, np.int32([left_line_window]), False, (-100, -100, 255), 30)
    cv2.polylines(out_img, np.int32([right_line_window]), False, (-100,-100, 255), 30)

    # get the middle points
    global mid_line_window
    mid_line_window = interpolate(left_line_window, right_line_window, poly_deg=2, n_points=720, plot=False)

    cv2.polylines(out_img, np.int32([mid_line_window]), False, (255, -100, -100), 30)


    unwarped = cv2.warpPerspective(out_img, M_inv, img_size , flags=cv2.INTER_LINEAR)

    result = cv2.addWeighted(img, 1, unwarped, 0.3, 0)
    cv2.putText(result,curvature_string , (100, 90), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), thickness=2)
    cv2.putText(result, offset_string, (100, 150), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), thickness=2)

    return result


In [6]:
def main():
    print("Running...")
    init = sl.InitParameters()
    cam = sl.Camera()
    if not cam.is_opened():
        print("Opening ZED Camera...")
    status = cam.open(init)
    if status != sl.ERROR_CODE.SUCCESS:
        print(repr(status))
        exit()

    runtime = sl.RuntimeParameters()
    mat = sl.Mat()
    point_cloud = sl.Mat()

    # lanenet
    input_tensor = tf.placeholder(dtype=tf.float32, shape=[1, 256, 512, 3], name='input_tensor')

    net = lanenet.LaneNet(phase='test', net_flag='vgg')
    binary_seg_ret, instance_seg_ret = net.inference(input_tensor=input_tensor, name='lanenet_model')

    postprocessor = lanenet_postprocess.LaneNetPostProcessor()

    saver = tf.train.Saver()

    # Set sess configuration
    sess_config = tf.ConfigProto()
    sess_config.gpu_options.per_process_gpu_memory_fraction = CFG.TEST.GPU_MEMORY_FRACTION
    sess_config.gpu_options.allow_growth = CFG.TRAIN.TF_ALLOW_GROWTH
    sess_config.gpu_options.allocator_type = 'BFC'

    sess = tf.Session(config=sess_config)


    # Main loop
    with sess.as_default():

        saver.restore(sess=sess, save_path="./model/tusimple_lanenet_vgg/tusimple_lanenet_vgg.ckpt")

        key = ''
        while key != 113:  # for 'q' key
            err = cam.grab(runtime)
            if err == sl.ERROR_CODE.SUCCESS:

                # Get ZED image
                cam.retrieve_image(mat, sl.VIEW.VIEW_LEFT)

                image = mat.get_data()[:,:,:3]

                # Get binary threshold
                t_start = time.time()
                image_resize = cv2.resize(image, (512, 256), interpolation=cv2.INTER_LINEAR)
                image_resize = image_resize / 127.5 - 1.0

                binary_seg_image, _ = sess.run( # ignore instance semgentation image
                    [binary_seg_ret, instance_seg_ret],
                    feed_dict={input_tensor: [image_resize]}
                )
                t_cost = time.time() - t_start

                # expected input to pipeline_final: 1280 * 720

                net_output = binary_seg_image[0].astype(np.uint8) * 255  # right from lanenet, no resize yet
                resized_net_output = cv2.resize(net_output, (1280, 720), interpolation=cv2.INTER_LINEAR)

                # Now getting the perspective transform matrix
                # Vertices extracted manually for performing a perspective transform
                bottom_left = [200,720]
                bottom_right = [1060, 720]
                top_left = [400, 500]
                top_right = [860, 500]

                source = np.float32([bottom_left,bottom_right,top_right,top_left])

                pts = np.array([bottom_left,bottom_right,top_right,top_left], np.int32)

                # Destination points are chosen such that straight lanes appear more or less parallel in the transformed image.
                bottom_left = [250,720]
                bottom_right = [1000, 720]
                top_left = [250, 1]
                top_right = [1000, 1]

                dst = np.float32([bottom_left,bottom_right,top_right,top_left])
                M = cv2.getPerspectiveTransform(source, dst)
                M_inv = cv2.getPerspectiveTransform(dst, source)

                result = pipeline_final(image, resized_net_output, M, M_inv)
                
                # subsample
                row_ids = [x for x in range(len(mid_line_window)) if x % 72 == 0]
                mid_line_window_sample = np.take(mid_line_window, row_ids, 0)
                mid_line_window_sample = mid_line_window_sample.astype(int)

                # get the point cloud
                cam.retrieve_measure(point_cloud, sl.MEASURE.MEASURE_XYZRGBA)
                try:
                    mid_line_coord = [utils.get_single_pt(point_cloud, pixel[0], pixel[1]) for pixel in mid_line_window_sample]
                    
                    # get reference x
                    ref_x = mid_line_window_sample[0][0]

                    # # get reference pixel arrays
                    ref_xy = mid_line_window_sample
                    ref_xy[:,0] = ref_x
                    ref_coord = [utils.get_single_pt(point_cloud, pixel[0], pixel[1]) for pixel in ref_xy]
                    diff_xyz = [0 if mid_line_coord[id] is None or ref_coord[id] is None else mid_line_coord[id][1] - ref_coord[id][1] for id in range(len(ref_coord))]
                    diff_xyz = np.nan_to_num(diff_xyz)


                    print("THIS IS THE DIFF YOU WANT", end = "\n")
                    print(diff_xyz, end="\n")
                    
                    # give the difference between the center and the lanes to MPC
                    mpc(diff_xyz)

                except:
                    continue
                cv2.imshow("result", result)
                # del result
                key = cv2.waitKey(1)
            else:
                key = cv2.waitKey(5)



    cv2.destroyAllWindows()
    sess.close()

    cam.close()
    print("\nFINISH")


In [ ]:
if __name__ == "__main__":
    main()